# Python for Ecologists

This is a brief review of Pandas data wrangling, working through the ecology data set from "Data Carpentry:  Python for Ecologists" http://www.datacarpentry.org/python-ecology/

## Imports

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ggplot import *
import sklearn as sk
from sklearn_pandas import DataFrameMapper, cross_val_score
import datetime as dt
from dateutil.parser import parse as date_parse

pd.options.display.max_columns = 50
pd.options.mode.chained_assignment = None

## Read Data

The ecology dataset I am using here consists of the species and weight of animals caught in plots in a study area. 

In [10]:
surveys_df_raw = pd.read_csv("https://ndownloader.figshare.com/files/2292172")

In [11]:
surveys_df_raw.head()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32,NaN
1,2,7,16,1977,3,NL,M,33,NaN
2,3,7,16,1977,2,DM,F,37,NaN
3,4,7,16,1977,7,DM,M,36,NaN
4,5,7,16,1977,3,DM,M,35,NaN


In [12]:
surveys_df_raw.describe()

,record_id,month,day,year,plot_id,hindfoot_length,weight
count,35549.000000,35549.000000,35549.000000,35549.000000,35549.000000,31438.000000,32283.000000
mean,17775.000000,6.474022,16.105966,1990.475231,11.397001,29.287932,42.672428
std,10262.256696,3.396583,8.256691,7.493355,6.799406,9.564759,36.631259
min,1.000000,1.000000,1.000000,1977.000000,1.000000,2.000000,4.000000
25%,8888.000000,4.000000,9.000000,1984.000000,5.000000,21.000000,20.000000
50%,17775.000000,6.000000,16.000000,1990.000000,11.000000,32.000000,37.000000
75%,26662.000000,9.000000,23.000000,1997.000000,17.000000,36.000000,48.000000
max,35549.000000,12.000000,31.000000,2002.000000,24.000000,70.000000,280.000000


So about 35,000 records of animals caught in different study plots.  

In [13]:
surveys_df_raw.dtypes

record_id            int64
month                int64
day                  int64
year                 int64
plot_id              int64
species_id          object
sex                 object
hindfoot_length    float64
weight             float64
dtype: object

## Handling Missing Values:  Imputation

Just from examining .head(), we can see that there are missing values, i.e., NaN in the dataframe.  There are many approaches to imputation of missing data.  The correct approach depends on the situation and what you will do with the data.  Here, I am replacing NaN with mean values.

In [14]:
surveys_df = surveys_df_raw

In [15]:
surveys_df['weight'] = surveys_df['weight'].fillna(surveys_df['weight'].mean())

NEED TO FILL IN OTHER NANS- HINDFOOT LENGTH ETC.

In [16]:
surveys_df.head()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32,42.672428
1,2,7,16,1977,3,NL,M,33,42.672428
2,3,7,16,1977,2,DM,F,37,42.672428
3,4,7,16,1977,7,DM,M,36,42.672428
4,5,7,16,1977,3,DM,M,35,42.672428


## Combining Dataframes:  Concatenation

In [19]:
# read in first 10 lines of surveys table
surveySub = surveys_df.head(10)
# grab the last 10 rows (minus the last one)
surveySubLast10 = surveys_df[-11:-1]
#reset the index values to the second dataframe appends properly
surveySubLast10=surveySubLast10.reset_index(drop=True)
# drop=True option avoids adding new index column with old index values

In [20]:
# stack the DataFrames on top of each other
verticalStack = pd.concat([surveySub, surveySubLast10], axis=0)

# place the DataFrames side by side
horizontalStack = pd.concat([surveySub, surveySubLast10], axis=1)

In [22]:
verticalStack

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32,42.672428
1,2,7,16,1977,3,NL,M,33,42.672428
2,3,7,16,1977,2,DM,F,37,42.672428
3,4,7,16,1977,7,DM,M,36,42.672428
4,5,7,16,1977,3,DM,M,35,42.672428
5,6,7,16,1977,1,PF,M,14,42.672428
6,7,7,16,1977,2,PE,F,NaN,42.672428
7,8,7,16,1977,1,DM,M,37,42.672428
8,9,7,16,1977,1,DM,F,34,42.672428
9,10,7,16,1977,6,PF,F,20,42.672428


In [23]:
horizontalStack

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32,42.672428,35539,12,31,2002,15,SF,M,26,68.000000
1,2,7,16,1977,3,NL,M,33,42.672428,35540,12,31,2002,15,PB,F,26,23.000000
2,3,7,16,1977,2,DM,F,37,42.672428,35541,12,31,2002,15,PB,F,24,31.000000
3,4,7,16,1977,7,DM,M,36,42.672428,35542,12,31,2002,15,PB,F,26,29.000000
4,5,7,16,1977,3,DM,M,35,42.672428,35543,12,31,2002,15,PB,F,27,34.000000
5,6,7,16,1977,1,PF,M,14,42.672428,35544,12,31,2002,15,US,NaN,NaN,42.672428
6,7,7,16,1977,2,PE,F,NaN,42.672428,35545,12,31,2002,15,AH,NaN,NaN,42.672428
7,8,7,16,1977,1,DM,M,37,42.672428,35546,12,31,2002,15,AH,NaN,NaN,42.672428
8,9,7,16,1977,1,DM,F,34,42.672428,35547,12,31,2002,10,RM,F,15,14.000000
9,10,7,16,1977,6,PF,F,20,42.672428,35548,12,31,2002,7,DO,M,36,51.000000


## Combining Dataframes:  Merge (Join)

Let's load some species data in another dataframe.

In [17]:
species_df = pd.read_csv('https://ndownloader.figshare.com/files/3299483',
                         keep_default_na=False, na_values=[""])

In [18]:
species_df.head()

,species_id,genus,species,taxa
0,AB,Amphispiza,bilineata,Bird
1,AH,Ammospermophilus,harrisi,Rodent
2,AS,Ammodramus,savannarum,Bird
3,BA,Baiomys,taylori,Rodent
4,CB,Campylorhynchus,brunneicapillus,Bird


In [24]:
speciesSub = species_df.head(10)

In [33]:
merged_inner = pd.merge(surveySub, species_df, on='species_id') # inner is default

In [34]:
merged_inner

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32,42.672428,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33,42.672428,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37,42.672428,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36,42.672428,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35,42.672428,Dipodomys,merriami,Rodent
5,8,7,16,1977,1,DM,M,37,42.672428,Dipodomys,merriami,Rodent
6,9,7,16,1977,1,DM,F,34,42.672428,Dipodomys,merriami,Rodent
7,6,7,16,1977,1,PF,M,14,42.672428,Perognathus,flavus,Rodent
8,10,7,16,1977,6,PF,F,20,42.672428,Perognathus,flavus,Rodent
9,7,7,16,1977,2,PE,F,NaN,42.672428,Peromyscus,eremicus,Rodent


In [35]:
# just for grins, here's a right join
pd.merge(surveySub, species_df, on='species_id', how='right').head(15)

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32,42.672428,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33,42.672428,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37,42.672428,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36,42.672428,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35,42.672428,Dipodomys,merriami,Rodent
5,8,7,16,1977,1,DM,M,37,42.672428,Dipodomys,merriami,Rodent
6,9,7,16,1977,1,DM,F,34,42.672428,Dipodomys,merriami,Rodent
7,6,7,16,1977,1,PF,M,14,42.672428,Perognathus,flavus,Rodent
8,10,7,16,1977,6,PF,F,20,42.672428,Perognathus,flavus,Rodent
9,7,7,16,1977,2,PE,F,NaN,42.672428,Peromyscus,eremicus,Rodent


In [37]:
# what if the columns to join on have different names?
surveySub_different = surveySub
surveySub_different = surveySub_different.rename(columns = {'species_id':'id'})
surveySub_different

,record_id,month,day,year,plot_id,id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32,42.672428
1,2,7,16,1977,3,NL,M,33,42.672428
2,3,7,16,1977,2,DM,F,37,42.672428
3,4,7,16,1977,7,DM,M,36,42.672428
4,5,7,16,1977,3,DM,M,35,42.672428
5,6,7,16,1977,1,PF,M,14,42.672428
6,7,7,16,1977,2,PE,F,NaN,42.672428
7,8,7,16,1977,1,DM,M,37,42.672428
8,9,7,16,1977,1,DM,F,34,42.672428
9,10,7,16,1977,6,PF,F,20,42.672428


In [40]:
merge_different = pd.merge(surveySub_different, species_df, left_on='id', right_on='species_id')
merge_different

,record_id,month,day,year,plot_id,id,sex,hindfoot_length,weight,species_id,genus,species,taxa
0,1,7,16,1977,2,NL,M,32,42.672428,NL,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33,42.672428,NL,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37,42.672428,DM,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36,42.672428,DM,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35,42.672428,DM,Dipodomys,merriami,Rodent
5,8,7,16,1977,1,DM,M,37,42.672428,DM,Dipodomys,merriami,Rodent
6,9,7,16,1977,1,DM,F,34,42.672428,DM,Dipodomys,merriami,Rodent
7,6,7,16,1977,1,PF,M,14,42.672428,PF,Perognathus,flavus,Rodent
8,10,7,16,1977,6,PF,F,20,42.672428,PF,Perognathus,flavus,Rodent
9,7,7,16,1977,2,PE,F,NaN,42.672428,PE,Peromyscus,eremicus,Rodent


In [43]:
merge_different.drop('id', 1, inplace=True)
merge_different

,record_id,month,day,year,plot_id,sex,hindfoot_length,weight,species_id,genus,species,taxa
0,1,7,16,1977,2,M,32,42.672428,NL,Neotoma,albigula,Rodent
1,2,7,16,1977,3,M,33,42.672428,NL,Neotoma,albigula,Rodent
2,3,7,16,1977,2,F,37,42.672428,DM,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,M,36,42.672428,DM,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,M,35,42.672428,DM,Dipodomys,merriami,Rodent
5,8,7,16,1977,1,M,37,42.672428,DM,Dipodomys,merriami,Rodent
6,9,7,16,1977,1,F,34,42.672428,DM,Dipodomys,merriami,Rodent
7,6,7,16,1977,1,M,14,42.672428,PF,Perognathus,flavus,Rodent
8,10,7,16,1977,6,F,20,42.672428,PF,Perognathus,flavus,Rodent
9,7,7,16,1977,2,F,NaN,42.672428,PE,Peromyscus,eremicus,Rodent


In [44]:
# a more succint merge
merged_inner2 = surveySub.merge(species_df, on='species_id')

In [45]:
merged_inner2

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32,42.672428,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33,42.672428,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37,42.672428,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36,42.672428,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35,42.672428,Dipodomys,merriami,Rodent
5,8,7,16,1977,1,DM,M,37,42.672428,Dipodomys,merriami,Rodent
6,9,7,16,1977,1,DM,F,34,42.672428,Dipodomys,merriami,Rodent
7,6,7,16,1977,1,PF,M,14,42.672428,Perognathus,flavus,Rodent
8,10,7,16,1977,6,PF,F,20,42.672428,Perognathus,flavus,Rodent
9,7,7,16,1977,2,PE,F,NaN,42.672428,Peromyscus,eremicus,Rodent
